In [1]:
import re
import ast
import logging
import requests
import pandas as pd
import time
import sys
import os
import json
import toml

from git_data_extractor import GitDataExtractor
sys.path.append(f'../utils')
from utils import adjust_message
import database as db

import yaml
import time
from openai import OpenAI
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

connect to CVEFIXes


In [2]:
df_master = pd.read_sql("SELECT * FROM cve ORDER BY modified DESC", con=db.conn)

In [3]:
# Domain sub-regex that supports various git hosting services
# Note: We're not including openssl in this list since it uses gitweb format
git_domain_regex = r'(?P<domain>(?:https|http)://(?:[a-zA-Z0-9-]+\.)*(?:bitbucket|github|gitlab|git|gitee|sr\.ht|torproject|gnome|redox-os)(?:\.[a-zA-Z0-9-]+)+)'

# Updated regex patterns using the shared domain sub-regex
# Note: We use raw f-strings (fr'') to properly escape the regex
git_url_commits = fr'(((?P<repo>{git_domain_regex}/(?P<owner>[^/]+)/(?P<project>[^/#?]+(?:\.git)?))/(\-/)?(commit|commits)/(?P<hash>[0-9a-fA-F]+)(?:[?#].*)?)+)'
git_url_issues = fr'(((?P<repo>{git_domain_regex}/(?P<owner>[^/]+)/(?P<project>[^/#?]+(?:\.git)?))/(\-/)?(issue|issues)/(?P<issueNum>\d+)(?:[?#].*)?)+)'
git_url_pulls = fr'(((?P<repo>{git_domain_regex}/(?P<owner>[^/]+)/(?P<project>[^/#?]+(?:\.git)?))/(pull|pulls)/(?P<pullNum>\d+)(?:[?#].*)?)+)'

In [ ]:
# openai_token = ''

# github_auth_token = ''
# tokens = {
#     "github.com": github_auth_token,
#     "gitlab.com": ''
# }

openai_client = OpenAI(
    base_url='https://api.cerebras.ai/v1',
    api_key=openai_token)

llm_model = "llama-3.3-70b"

### Commit, PR, issue data structure
We require:
1. Vulnerability identifier (CVE ID in our case)
2. Repo address, as a URL or author-project combination
3. Unique identifier of data

#### Commit
Each row is a list of:
1. CVE ID
2. Hash
3. Repo URL

#### Pull Request (PR)
1. CVE ID
2. Pull request URL
3. Repo URL

Currently, some scripts don't follow (2).

#### Issue
1. CVE ID
2. Issue URL
3. Repo URL

In [5]:
logging.info('Extracting all the reference urls from CVE...')

master_commits = list()
master_prs = list()
master_issues = list()
master_raws = list()
master_invalid_refs = list()

for i in range(len(df_master)):
    ref_list = ast.literal_eval(df_master['references'].iloc[i])
    ref_list = list(set(ref_list))
    # print(ref_list)
    link = False
    logmsg = None
    cve_id = df_master['id'][i]
    for url in ref_list:
        if link := re.search(git_url_commits, url):
            master_commits.append(tuple([
                cve_id, link.group('hash'), link.group('repo').replace(r'http:', r'https:')]))
            logmsg = "commits"
    if not logmsg:
        for url in ref_list:
            if link := re.search(git_url_pulls, url):
                master_prs.append(tuple([
                    cve_id, url,
                    link.group('repo').replace(r'http:', r'https:')]))
                logmsg = "pulls"
    if not logmsg:
        for url in ref_list:
            if link := re.search(git_url_issues, url):
                master_issues.append(tuple([
                    cve_id, url, link.group('repo').replace(r'http:', r'https:')]))
                logmsg = "issues"
    if not logmsg:
        for url in ref_list:
            if "raw.githubusercontent.com/rustsec/advisory-db/main/crates" in url:
                master_raws.append([cve_id, url])
                logmsg = "raw"
    if not logmsg:
        master_invalid_refs.append(ref_list)

print("# commits: "+str(len(master_commits)))
print("# pulls: "+str(len(master_prs)))
print("# issues: "+str(len(master_issues)))
print("# raw: "+str(len(master_raws)))
print("# invalid ref: "+str(len(master_invalid_refs)))

02/01/2026 21:56:32 INFO: Extracting all the reference urls from CVE...


# commits: 593
# pulls: 199
# issues: 381
# raw: 4
# invalid ref: 211


### Get commit/PR/issue from raw

In [6]:
# Code intended for raw types (few vulnerabilities)

def get_fix_from_raw(url):
    headers = {'Authorization': 'token %s' % github_auth_token}
    logging.info("About to get CVE related pulls or commits or issues in {}".format(url))
    sleep_cnt = 0
    while(True):
        try:
            response = requests.get(url, headers=headers)
            fix = response.text.split("\n", 1)[1]
            fix = fix.split("```")[0]
            parsed_toml = toml.loads(fix)
            # print(parsed_toml)
            if 'url' in parsed_toml['advisory']:
                url = parsed_toml['advisory']['url']
            else:
                url = None
            return url
        except Exception as e:
            if(sleep_cnt >= 8):
                break
            logging.info("sleep.....")
            sleep_cnt += 1
            time.sleep(5*sleep_cnt)

count = 0
issues_with_solutions = list()
for raw_tmp in master_raws:
    # print(cnt)
    cve_id = raw_tmp[0]
    url = raw_tmp[1]
    if url:= get_fix_from_raw(url):
        # print(url)
        issues_with_solutions.append([cve_id, url])
    count += 1
    
for fix_url in issues_with_solutions:
    cve_id = fix_url[0]
    url = fix_url[1]
    if link := re.search(git_url_commits, url):
        master_commits.append([cve_id, link.group('hash'), link.group('repo').replace(r'http:', r'https:')])
        logmsg = "commits"
    elif link := re.search(git_url_pulls, url):
        master_prs.append([cve_id, [link.group('owner'), link.group('project'), link.group('pullNum')], link.group('repo').replace(r'http:', r'https:')])
        logmsg = "pulls"
    elif link := re.search(git_url_issues, url):
        master_issues.append([cve_id, url, link.group('repo').replace(r'http:', r'https:')])
        logmsg = "issues"

02/01/2026 21:56:32 INFO: About to get CVE related pulls or commits or issues in https://raw.githubusercontent.com/rustsec/advisory-db/main/crates/ckb/RUSTSEC-2021-0109.md
02/01/2026 21:56:33 INFO: About to get CVE related pulls or commits or issues in https://raw.githubusercontent.com/rustsec/advisory-db/main/crates/ckb/RUSTSEC-2021-0108.md
02/01/2026 21:56:33 INFO: About to get CVE related pulls or commits or issues in https://raw.githubusercontent.com/rustsec/advisory-db/main/crates/ckb/RUSTSEC-2021-0107.md
02/01/2026 21:56:34 INFO: About to get CVE related pulls or commits or issues in https://raw.githubusercontent.com/rustsec/advisory-db/main/crates/libpulse-binding/RUSTSEC-2018-0021.md


### Get commits/pull requests from issues

In [7]:
logging.info(f"Processing {len(master_issues)} issues...")
count = 0
failed_count = 0
issues_with_solutions = {}
issues_without_solutions = {}

02/01/2026 21:56:34 INFO: Processing 381 issues...


In [8]:
def is_fix_commits(msg):
    bugfixing_keywords = {'fix', 'repair', 'error', 'bug', 'issue', 'exception', 'cve', 'prevent'}
    exclude_keywords = {'readme', 'test', 'typo', 'cleanup', 'bump', 'merge'}   
    msg = adjust_message(msg).lower()
    for t in exclude_keywords:
        if (t in msg):
            return False
    for k in bugfixing_keywords:
        if k in msg:
            return True
    return False


def issues_load_solution_cache(cache_path):
    """Loads solutions from cache if available."""
    file_path = os.path.join(cache_path, "solutions.json")
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return json.load(f)
    return None

def issues_save_solution_cache(cache_path, solutions):
    """Saves solutions to cache."""
    file_path = os.path.join(cache_path, "solutions.json")
    # Ensure directory exists
    os.makedirs(cache_path, exist_ok=True)
    with open(file_path, 'w') as f:
        json.dump(solutions, f)

### Find solutions (PR and commit links) in issues

In [9]:
patterns_per_type = {
    'commit_link': re.compile(fr'(?P<repo>{git_domain_regex}/(?P<owner>[^/]+)/(?P<project>[^/#?]+(?:\.git)?))/(\-/)?(commit|commits)/(?P<hash>[0-9a-fA-F]+)(?:[?#].*)?', re.IGNORECASE),
    'pr_link': re.compile(fr'(?P<repo>{git_domain_regex}/(?P<owner>[^/]+)/(?P<project>[^/#?]+(?:\.git)?))/(pull|pulls)/(?P<pullNum>\d+)(?:[?#].*)?', re.IGNORECASE)
}

def issues_heuristic_search(data, extractor: GitDataExtractor):
    """
    Recursively searches a dictionary/list structure (Issue/PR data) 
    for PR and Commit URLs using the defined regexes.
    Ignores matches containing 'advisory'.
    
    Args:
        data (dict/list): The raw issue data to traverse.
        
    Returns:
        list: A list of dictionaries, where each dict is a match 
                (with keys: type, repo, owner, project, hash/issueNum/pullNum, etc.)
    """
    results = []
    ignore_keywords = ['advisory', 'advisories', 'adv']

    def _traverse(obj):
        if isinstance(obj, str):
            # Apply regexes to the string
            for p_type, regex in patterns_per_type.items():
                for match in regex.finditer(obj):
                    full_match = match.group(0)
                    
                    # Filter out advisories
                    if any(word in full_match.lower() for word in ignore_keywords):
                        continue

                    commit = extractor.get_commit_by_hash(match.group('hash'))
                    commit_msg = commit.get('message')

                    # If we found a commit message, check if it's a fix
                    if commit_msg and is_fix_commits(commit_msg):
                        match_dict = match.groupdict()
                        match_dict['type'] = p_type
                        match_dict['url'] = full_match
                        results.append(match_dict)
        elif isinstance(obj, dict):
            for val in obj.values():
                _traverse(val)
        elif isinstance(obj, list):
            for item in obj:
                _traverse(item)

    _traverse(data)
    return results

def issues_llm_search(issue: dict, client, model):
    """Invokes OpenAI to find solutions using JSON dumps of the dataframes."""

    system_prompt = """
        You are an expert software maintainer analyzing a GitHub Issue.
        You will receive the issue metadata and activity history in YAML format.
        
        Your goal is to identify how the issue was solved. Possible solutions are: a PR link, a commit, a tag, stating that the project is unmaintained, abandoned, or replaced or superceded, or no solution.
        
        Identify the solution type from the following:
        1. 'pr_link': A specific Pull Request link that fixes the issue. MUST be a URL.
        2. 'commit_link': A specific commit hash or link that fixes the issue. MUST be a URL.
        3. 'status': The issue states the project is unmaintained, deprecated, or replaced.
        
        The output HAS TO contain ONLY the YAML object, no other strings before or after it. Do not wrap the output in markdown code blocks.
        Output format:
        solutions:
            - type: pr_link
            value: https://github.com/owner/repo/pull/123
            - type: status
            value: unmaintained
    """

    try:
        issue_json = yaml.dump(issue, Dumper=Dumper)

        user_content = f"""
        Issue Data (YAML):
        {issue_json}
        """

        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ],
            temperature=0.0
        )
        
        content = response.choices[0].message.content
        data = yaml.load(content, Loader=Loader)
        return data.get("solutions", [])
        
    except Exception as e:
        logging.error(f"LLM search failed")
        logging.error(e)
        return []

logging.info(f"Processing {len(master_issues)} issues...")
count = 0
failed_count = 0

for issue in master_issues:
    count += 1
    
    cve_id = issue[0]
    issue_url = issue[1]
    repo_url = issue[2]

    # print("Processing " + issue_url)

    try:
        # 1. Extract Dataframes
        extractor = GitDataExtractor(issue_url, tokens)
        issue_data, _ = extractor.extract()
        assert(issue_data is not None)
        
        if not issue_data:
            logging.warning(f"No data extracted for {issue_url}")
            continue

        # 2. Find Solutions
        cache_path = extractor._get_cache_path()
        solutions = []

        # Note for future: using redis would be easier
        cached_solutions = issues_load_solution_cache(cache_path)
        if cached_solutions is not None:
            solutions = cached_solutions
        else:
            solutions.extend(issues_heuristic_search(issue_data, extractor))
        
            if len(solutions) == 0 and openai_client:
                time.sleep(2)
                solutions.extend(issues_llm_search(issue_data, openai_client, llm_model))
            
            issues_save_solution_cache(cache_path, solutions)
        
        # 3. Process Results
        if solutions and len(solutions) > 0:
            issues_with_solutions[issue] = solutions
        else:
            issues_without_solutions[issue] = solutions

    except Exception as e:
        failed_count += 1

    if count % 50 == 0:
        logging.info(f"Processed {count} issues. Fixes found: {len(issues_with_solutions)}")

print(f"Finished. Failed count: {failed_count}")
print(f"Total Fixes found: {len(issues_with_solutions)}")

02/01/2026 21:56:38 INFO: Processing 381 issues...


02/01/2026 21:56:57 INFO: Processed 50 issues. Fixes found: 45
02/01/2026 21:56:57 INFO: Processed 100 issues. Fixes found: 91
02/01/2026 21:56:57 INFO: Processed 150 issues. Fixes found: 137
02/01/2026 21:58:30 INFO: Processed 200 issues. Fixes found: 186
02/01/2026 21:58:33 INFO: Processed 250 issues. Fixes found: 228
02/01/2026 21:58:35 INFO: Processed 300 issues. Fixes found: 275
02/01/2026 21:58:36 INFO: Processed 350 issues. Fixes found: 322


Finished. Failed count: 12
Total Fixes found: 352


In [10]:
print(f"Processing {len(issues_with_solutions)} issues with solutions to extract PR and commit links...")

prs_from_issues = []
commits_from_issues = []
failed_count = 0

for issue, solutions in issues_with_solutions.items():
    cve_id = issue[0]
    issue_url = issue[1]
    repo_url = issue[2]
    for solution in solutions:
        try:
            if solution.get('type') == 'pr_link':
                url = solution.get('url')
                if url is None:
                    continue
                if link := re.search(git_url_pulls, url):
                    if len(link.group('owner')) > 0 and len(link.group('project')) > 0 and len(link.group('pullNum')) > 0:
                        prs_from_issues.append(tuple([cve_id, url, repo_url]))
                    # TODO: fix regex not stripping what is after the link
                    if "#" in url:
                        pass
            elif solution.get('type') == 'commit_link':
                url = solution.get('url')
                if url is None:
                    continue
                if link := re.search(git_url_commits, url):
                    if len(link.group('owner')) > 0 and len(link.group('project')) > 0 and len(link.group('hash')) > 0:
                        commits_from_issues.append(tuple([cve_id, link.group('hash'), repo_url]))
        except IndexError as e:
            failed_count += 1

print(f"Detected {len(prs_from_issues)} PR links from issue solutions")
print(f"Detected {len(commits_from_issues)} commit links from issue solutions")
print(f"We had {failed_count} failures")

Processing 352 issues with solutions to extract PR and commit links...
Detected 159 PR links from issue solutions
Detected 671 commit links from issue solutions
We had 0 failures


### Find useful commits in the dataset PRs

In [11]:
# Process PR links from the df_master dataset (vuls -> references, pulls_from_master)
# to extract commits using GitDataExtractor
print(f"\nExtracting commits from {len(master_prs)} PR links found in issues...")

commits_from_db_prs = []
failed_count = 0
processed_count = 0

for pr_entry in master_prs:
    processed_count += 1
    if processed_count % 10 == 0:
        print(f"  Processed {processed_count}/{len(master_prs)} PRs...")
    
    cve_id = pr_entry[0]
    pr_url = pr_entry[1]
    repo_url = pr_entry[2]
    
    try:
        extractor = GitDataExtractor(pr_url, tokens)
        
        # Get commits from PR using the new method
        commits = extractor.get_pr_commits()
        
        if not commits:
            print(f"    No commits found for {pr_url}")
            continue
        
        # Filter for fix commits
        fix_commits = []
        for commit in commits:
            if is_fix_commits(commit['message']):
                fix_commits.append(commit)
        
        # Add to results
        for commit in fix_commits:
            commits_from_db_prs.append(tuple([cve_id, commit['hash'], repo_url]))
        
        # Rate limiting
        time.sleep(0.5)
        
    except Exception as e:
        print(f"    Error processing {pr_url}: {str(e)[:100]}...")
        failed_count += 1

print(f"\nFound {len(commits_from_db_prs)} fix commits from {len(master_prs) - failed_count} PRs")
print(f"Failed to process {failed_count} PRs")


Extracting commits from 199 PR links found in issues...
  Processed 10/199 PRs...
  Processed 20/199 PRs...
  Processed 30/199 PRs...
  Processed 40/199 PRs...
    No commits found for https://github.com/gwenn/lemon-rs/pull/8
  Processed 50/199 PRs...
  Processed 60/199 PRs...
  Processed 70/199 PRs...
  Processed 80/199 PRs...
  Processed 90/199 PRs...
    No commits found for https://github.com/bluejekyll/trust-dns/pull/1952
  Processed 100/199 PRs...
  Processed 110/199 PRs...
  Processed 120/199 PRs...
  Processed 130/199 PRs...
  Processed 140/199 PRs...
  Processed 150/199 PRs...
    No commits found for https://github.com/RustCrypto/stream-ciphers/pull/64
  Processed 160/199 PRs...
  Processed 170/199 PRs...
    No commits found for https://github.com/interledger-rs/interledger-rs/pull/744
  Processed 180/199 PRs...
    No commits found for https://github.com/maidsafe/sn_api/pull/623
    No commits found for https://github.com/maidsafe/sn_api/pull/619
    No commits found for h

### Find useful commits in issue PRs

In [12]:
# Process PR links from issues to extract commits using GitDataExtractor
print(f"\nExtracting commits from {len(prs_from_issues)} PR links found in issues...")

commits_from_issue_prs = []
failed_count = 0
processed_count = 0

for pr in prs_from_issues:
    cve_id = pr[0]
    pr_url = pr[1]
    repo_url = pr[2]
    try:
        processed_count += 1
        if processed_count % 10 == 0:
            print(f"  Processed {processed_count}/{len(prs_from_issues)} PRs...")
        
        extractor = GitDataExtractor(pr_url, tokens)
        
        # Get commits from PR using the new method
        commits = extractor.get_pr_commits()
        
        if not commits:
            print(f"    No commits found for {pr_url}")
            continue
        
        # Filter for fix commits
        fix_commits = []
        for commit in commits:
            if is_fix_commits(commit['message']):
                fix_commits.append(commit)
        
        # Add to results
        for commit in fix_commits:
            commits_from_issue_prs.append(tuple([cve_id, commit['hash'], repo_url]))
        
        # Rate limiting
        time.sleep(0.5)
        
    except Exception as e:
        print(f"    Error processing {pr_url}: {str(e)[:100]}...")
        failed_count += 1

print(f"\nFound {len(commits_from_issue_prs)} fix commits from {len(prs_from_issues) - failed_count} PRs")
print(f"Failed to process {failed_count} PRs")


Extracting commits from 159 PR links found in issues...
  Processed 10/159 PRs...
  Processed 20/159 PRs...
  Processed 30/159 PRs...
  Processed 40/159 PRs...
  Processed 50/159 PRs...
  Processed 60/159 PRs...
  Processed 70/159 PRs...
  Processed 80/159 PRs...
  Processed 90/159 PRs...
  Processed 100/159 PRs...
  Processed 110/159 PRs...
  Processed 120/159 PRs...
  Processed 130/159 PRs...
  Processed 140/159 PRs...
  Processed 150/159 PRs...

Found 162 fix commits from 159 PRs
Failed to process 0 PRs


In [17]:
# Combine multiple sources of commits
print("Combining all commit sources...")

# Ensure all variables exist (some might be undefined if previous cells failed)
all_commits = []

# Add commits from each source with source tracking
if 'master_commits' in locals() or 'master_commits' in globals():
    # Convert all to tuples for consistency
    for commit in master_commits:
        if isinstance(commit, (list, tuple)) and len(commit) >= 3:
            cve_id = commit[0]
            commit_hash = commit[1]
            repo_url = commit[2]
            all_commits.append((cve_id, commit_hash, repo_url, 'master'))
    print(f"  Added {len(master_commits)} commits from master_commits")

if 'commits_from_issues' in locals() or 'commits_from_issues' in globals():
    for commit in commits_from_issues:
        if isinstance(commit, (list, tuple)) and len(commit) >= 3:
            cve_id = commit[0]
            commit_hash = commit[1]
            repo_url = commit[2]
            all_commits.append((cve_id, commit_hash, repo_url, 'issue_links'))
    print(f"  Added {len(commits_from_issues)} commits from commit_links_from_issues")

if 'commits_from_db_prs' in locals() or 'commits_from_db_prs' in globals():
    for commit in commits_from_db_prs:
        if isinstance(commit, (list, tuple)) and len(commit) >= 3:
            cve_id = commit[0]
            commit_hash = commit[1]
            repo_url = commit[2]
            all_commits.append((cve_id, commit_hash, repo_url, 'db_prs'))
    print(f"  Added {len(commits_from_db_prs)} commits from commits_from_db_prs")

if 'commits_from_issue_prs' in locals() or 'commits_from_issue_prs' in globals():
    for commit in commits_from_issue_prs:
        if isinstance(commit, (list, tuple)) and len(commit) >= 3:
            cve_id = commit[0]
            commit_hash = commit[1]
            repo_url = commit[2]
            all_commits.append((cve_id, commit_hash, repo_url, 'issue_prs'))
    print(f"  Added {len(commits_from_issue_prs)} commits from commits_from_issue_prs")

print(f"\nTotal commits before deduplication: {len(all_commits)}")

# Remove duplicates based on hash only (collisions are rare)
seen_hashes = set()
unique_commits = []
for cve_id, commit_hash, repo_url, source in all_commits:
    if commit_hash not in seen_hashes:
        seen_hashes.add(commit_hash)
        unique_commits.append((cve_id, commit_hash, repo_url, source))

print(f"Unique commits after deduplication: {len(unique_commits)}")
print(f"Removed {len(all_commits) - len(unique_commits)} duplicates")

# Create DataFrame
df_combined = pd.DataFrame(unique_commits, columns=['cve_id', 'hash', 'repo_url', 'source'])
df_combined = df_combined[['cve_id', 'hash', 'repo_url']]  # Drop source column for final output

# Save to CSV
df_combined.to_csv('fix_commits.csv', index=False)
print(f"\nSaved {len(df_combined)} unique commits to fix_commits.csv")

# Show summary by source
print("\nSummary by source:")
source_counts = {}
for _, _, _, source in unique_commits:
    source_counts[source] = source_counts.get(source, 0) + 1

for source, count in source_counts.items():
    print(f"  {source}: {count} commits")

# Display first few rows
print("\nFirst 5 commits:")
print(df_combined.head())

Combining all commit sources...
  Added 593 commits from master_commits
  Added 671 commits from commit_links_from_issues
  Added 185 commits from commits_from_db_prs
  Added 162 commits from commits_from_issue_prs

Total commits before deduplication: 1611
Unique commits after deduplication: 1308
Removed 303 duplicates

Saved 1308 unique commits to fix_commits.csv

Summary by source:
  master: 559 commits
  issue_links: 542 commits
  db_prs: 136 commits
  issue_prs: 71 commits

First 5 commits:
                                              cve_id  \
0                            ['GHSA-95qg-89c2-w5hj']   
1  ['GHSA-c86p-w88r-qvqr', 'GHSA-4p46-pwfr-66x6',...   
2       ['RUSTSEC-2025-0138', 'GHSA-8vxj-4cph-c596']   
3       ['RUSTSEC-2023-0095', 'GHSA-gpcv-p28p-fv2p']   
4       ['GHSA-vx24-x4mv-vwr5', 'RUSTSEC-2024-0446']   

                                       hash  \
0  8e0b565e7876a83b0e1cfbacb8af39dadfdcc500   
1  ec2d3cf1d91f148c84e4806b4f0b3c98f6df3b38   
2  31a97803995bd946295

### !!! Note
We export the closed issues without fix prs/commits to `manual_issues.csv`.
Here we manually checked these issues and found their fix commits/prs if any.


In [15]:
import yaml

print(f"Dumping {len(issues_without_solutions)} issues without solutions to manual_issues.yaml")

with open('manual_issues.yaml', 'w') as f:
    yaml.dump(issues_without_solutions, f, default_flow_style=False, sort_keys=False)

print("Done writing to manual_issues.yaml")


Dumping 17 issues without solutions to manual_issues.yaml
Done writing to manual_issues.yaml


### Add Manual Fix commits/PRs from issues

In [ ]:
import math
df_closed_issue = pd.read_csv("manual_issues.csv")
for index, issue_tuple in df_closed_issue.iterrows():
    cve_id = issue_tuple['cve_id']
    url = issue_tuple['fix']
    if not url or math.isnan(url): 
        continue
    if link := re.search(git_url_commits, url):
        master_commits.append([cve_id, link.group('hash'), link.group('repo').replace(r'http:', r'https:')])
        logmsg = "commits"
    elif link := re.search(git_url_pulls, url):
        master_prs.append([cve_id, [link.group('owner'), link.group('project'), link.group('pullNum')], link.group('repo').replace(r'http:', r'https:')])
        logmsg = "pulls"

### Deal with merge commits

In [24]:
import pandas as pd
from pydriller import Repository, Git
import logging
import os
import re
import database as db
from utils import get_full_project_name, is_git_repo

dest = "/home/dev/Code/rust_ecosystem/repos_mirror"

fixes = pd.read_csv("fix_commits.csv")
# df_fixes.drop_duplicates(subset =['hash', 'repo_url'], keep = 'first', inplace = True)
df_result = pd.DataFrame(columns=fixes.columns)

for index, issue_tuple in fixes.iterrows():
    repo_url = issue_tuple["repo_url"]
    full_project_name = get_full_project_name(repo_url)
    repo_dest_path = os.path.join(dest, full_project_name)
    if os.path.exists(repo_dest_path):
        if is_git_repo(repo_dest_path):
            commit_hash = issue_tuple["hash"]
            cve_id = issue_tuple["cve_id"]
            try:
                commit = Git(repo_dest_path).get_commit(commit_hash)
                if commit.merge:
                    for parent in commit.parents:
                        commit_hash = parent
                        commit = Git(repo_dest_path).get_commit(commit_hash)
                        if not commit.merge:
                            if is_fix_commits(commit.msg):
                                df_result.loc[len(df_result)] = [cve_id, commit_hash, repo_url]
                else:
                    df_result.loc[len(df_result)] = issue_tuple
                
            except Exception as e:
                # "This commit does not belong to any branch on this repository, and may belong to a fork outside of the repository.
                print(e)
                logging.warning(f'Could not retrieve commit information from: {repo_dest_path}\n')
    else:
        logging.info(f"Repository {repo_url} not cloned. Skipping.")

02/02/2026 02:52:01 INFO: Repository https://github.com/jtesta/ssh-audit not cloned. Skipping.
02/02/2026 02:52:01 INFO: Repository https://github.com/mscdex/ssh2 not cloned. Skipping.
02/02/2026 02:52:01 INFO: Repository https://github.com/TeraTermProject/teraterm not cloned. Skipping.
02/02/2026 02:52:01 INFO: Repository https://github.com/golang/crypto not cloned. Skipping.
02/02/2026 02:52:01 INFO: Repository https://github.com/connectbot/sshlib not cloned. Skipping.
02/02/2026 02:52:01 INFO: Repository https://github.com/raviqqe/array-queue not cloned. Skipping.
02/02/2026 02:52:02 INFO: Repository https://github.com/sfackler/rust-openssl not cloned. Skipping.
02/02/2026 02:52:02 INFO: Repository https://git.distrust.co/public/blahaj not cloned. Skipping.
02/02/2026 02:52:02 INFO: Repository https://github.com/sfackler/rust-openssl not cloned. Skipping.
02/02/2026 02:52:02 INFO: Repository https://github.com/stepancheg/rust-protobuf not cloned. Skipping.
02/02/2026 02:52:02 INFO: 

SHA b'8a8814e7cc97d44f4d194586b3e9deb904c99488' could not be resolved, git returned: b'8a8814e7cc97d44f4d194586b3e9deb904c99488 missing'


02/02/2026 02:52:02 INFO: Repository https://github.com/CloudCannon/pagefind not cloned. Skipping.
02/02/2026 02:52:02 INFO: Repository https://github.com/memorysafety/sudo-rs not cloned. Skipping.
02/02/2026 02:52:02 INFO: Repository https://github.com/openvm-org/openvm not cloned. Skipping.
02/02/2026 02:52:03 INFO: Repository https://github.com/sfackler/rust-openssl not cloned. Skipping.
02/02/2026 02:52:03 INFO: Repository https://github.com/rustsec/advisory-db not cloned. Skipping.
02/02/2026 02:52:03 INFO: Repository https://github.com/netvl/xml-rs not cloned. Skipping.
02/02/2026 02:52:03 INFO: Repository https://github.com/00xc/xml-rs not cloned. Skipping.
02/02/2026 02:52:03 INFO: Repository https://github.com/netvl/xml-rs not cloned. Skipping.
02/02/2026 02:52:03 INFO: Repository https://github.com/openssl/openssl not cloned. Skipping.
02/02/2026 02:52:03 INFO: Repository https://github.com/wasm3/wasm3 not cloned. Skipping.
02/02/2026 02:52:03 INFO: Repository https://github.

SHA b'8b9eab05795fdc098976262853b2498055c7a8f3' could not be resolved, git returned: b'8b9eab05795fdc098976262853b2498055c7a8f3 missing'


02/02/2026 02:52:04 INFO: Repository https://github.com/paritytech/ink not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/alexcrichton/bzip2-rs not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/capnproto/capnproto not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/conduit-rust/conduit-hyper not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/rust-osdev/linked-list-allocator not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/rulex-rs/rulex not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/rulex-rs/rulex not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/waycrate/swhkd not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/waycrate/swhkd not cloned. Skipping.
02/02/2026 02:52:04 INFO: Repository https://github.com/waycrate/swhkd not cloned. Skipping.
02/02/2026 02:52:04 INFO: Re

...

SHA b'b6051ec01bd584f888f85bc8ef2dc0caa91fda67' could not be resolved, git returned: b'b6051ec01bd584f888f85bc8ef2dc0caa91fda67 missing'
SHA b'7ad478a62b9412aba808eb1f737db16dea6de5cc' could not be resolved, git returned: b'7ad478a62b9412aba808eb1f737db16dea6de5cc missing'
SHA b'8b2734351a5cd05a9bd3315f1af76c28b5852fec' could not be resolved, git returned: b'8b2734351a5cd05a9bd3315f1af76c28b5852fec missing'
SHA b'433dbb31bfb488caecf52c8a4695f2b9b19c6af5' could not be resolved, git returned: b'433dbb31bfb488caecf52c8a4695f2b9b19c6af5 missing'
SHA b'1af607a8e2a8273fd2a6b6dbc5a1830ba7882fca' could not be resolved, git returned: b'1af607a8e2a8273fd2a6b6dbc5a1830ba7882fca missing'
SHA b'bc484094c7b9d5a16fe01df6ddd0aa6a3f451335' could not be resolved, git returned: b'bc484094c7b9d5a16fe01df6ddd0aa6a3f451335 missing'
SHA b'268a58e53a16788f0762802dd3510c5d21d585ff' could not be resolved, git returned: b'268a58e53a16788f0762802dd3510c5d21d585ff missing'
SHA b'a37485f0ad61c34fef643df38f79739038a

02/02/2026 02:52:09 INFO: Repository https://github.com/rustsec/advisory-db not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/polytope-labs/ismp-substrate not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/polytope-labs/ismp-substrate not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/polytope-labs/ismp-substrate not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/polytope-labs/ismp-substrate not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/RustyYato/cell-project not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/RustyYato/cell-project not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/sfackler/rust-openssl not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/lz4/lz4 not cloned. Skipping.
02/02/2026 02:52:09 INFO: Repository https://github.com/getzola/zola not cloned. Skipp

SHA b'f0d3b0879322983b1c815ee70f7bf4d2b5592ed7' could not be resolved, git returned: b'f0d3b0879322983b1c815ee70f7bf4d2b5592ed7 missing'
SHA b'b6bb500b74078e06ee95b369b86203cc9f558325' could not be resolved, git returned: b'b6bb500b74078e06ee95b369b86203cc9f558325 missing'
SHA b'fad6b6df6a3f26927abc65de2de09a5820491cf2' could not be resolved, git returned: b'fad6b6df6a3f26927abc65de2de09a5820491cf2 missing'


02/02/2026 02:52:10 INFO: Repository https://github.com/rustsec/advisory-db not cloned. Skipping.
02/02/2026 02:52:10 INFO: Repository https://github.com/rustsec/advisory-db not cloned. Skipping.
02/02/2026 02:52:10 INFO: Repository https://github.com/rustsec/advisory-db not cloned. Skipping.
02/02/2026 02:52:10 WARNING: Could not retrieve commit information from: /home/dev/Code/rust_ecosystem/repos_mirror/deprecrated_net2-rs

02/02/2026 02:52:10 WARNING: Could not retrieve commit information from: /home/dev/Code/rust_ecosystem/repos_mirror/deprecrated_net2-rs

02/02/2026 02:52:10 WARNING: Could not retrieve commit information from: /home/dev/Code/rust_ecosystem/repos_mirror/RustCrypto_RSA

02/02/2026 02:52:10 INFO: Repository https://github.com/rust-lang/rust not cloned. Skipping.
02/02/2026 02:52:10 INFO: Repository https://github.com/joshua-maros/ouroboros not cloned. Skipping.
02/02/2026 02:52:10 INFO: Repository https://github.com/joshua-maros/ouroboros not cloned. Skipping.
02/02

SHA b'a344d10810187430811b43a71ad9b742d25f9b37' could not be resolved, git returned: b'a344d10810187430811b43a71ad9b742d25f9b37 missing'
SHA b'ba9a2f9a810cea096d186a4bad7763e1ba8369ea' could not be resolved, git returned: b'ba9a2f9a810cea096d186a4bad7763e1ba8369ea missing'
SHA b'ef837b8cea0b9fc1b5d89bb5965bc6c51cae1cad' could not be resolved, git returned: b'ef837b8cea0b9fc1b5d89bb5965bc6c51cae1cad missing'
SHA b'6fd909b2b1dd2789e0f1af79283c6105fa502ca3' could not be resolved, git returned: b'6fd909b2b1dd2789e0f1af79283c6105fa502ca3 missing'
SHA b'b133d6776fde22e944eb7f266ee165ffcf7cdb09' could not be resolved, git returned: b'b133d6776fde22e944eb7f266ee165ffcf7cdb09 missing'
SHA b'dc4807182337d44a591d567292b360ca494a325c' could not be resolved, git returned: b'dc4807182337d44a591d567292b360ca494a325c missing'
SHA b'2ad7310e04784f468f7c42242b1db2b33fcae946' could not be resolved, git returned: b'2ad7310e04784f468f7c42242b1db2b33fcae946 missing'
SHA b'5292db8522c7810c50808df417eac71f656

In [25]:
df_result.to_csv('fix_commits2.csv')
df_result

,cve_id,hash,repo_url
0,"['GHSA-c86p-w88r-qvqr', 'GHSA-4p46-pwfr-66x6',...",ec2d3cf1d91f148c84e4806b4f0b3c98f6df3b38,https://github.com/briansmith/ring
1,"['RUSTSEC-2025-0138', 'GHSA-8vxj-4cph-c596']",31a97803995bd94629528ba841b2418d3ca01860,https://github.com/denoland/deno
2,"['RUSTSEC-2023-0095', 'GHSA-gpcv-p28p-fv2p']",c1bc4ed71dcc9842b7dc1ea26f278f105074bbaa,https://github.com/cloudflare/odoh-rs
3,"['GHSA-vx24-x4mv-vwr5', 'RUSTSEC-2024-0446']",cfc58161e0ec595db90af686ad77a73df6d44d74,https://github.com/starship/starship
4,['GHSA-hxf5-99xg-86hw'],dcc3818039761331fbeacbb3a40c542b65b5ebf7,https://github.com/bytecodealliance/cap-std
...,...,...,...
716,['GHSA-45w3-v3g4-54pm'],89de1b24d6fd196a14fcad5761c54ada60f3bacc,https://github.com/parallaxsecond/parsec
717,['GHSA-45w3-v3g4-54pm'],69642f60eb50687282c84c6920f78e91058c5eee,https://github.com/parallaxsecond/parsec
718,['GHSA-45w3-v3g4-54pm'],e6e97db397d927b3d4a4a8d0d5ad46226fb789d5,https://github.com/parallaxsecond/parsec
719,['GHSA-gjrj-9rj4-pgwx'],9edd9e1e1811f3fd757f458fbe84508472833913,https://github.com/actix/actix-web


In [26]:
df = pd.read_csv('fix_commits2.csv')
df.to_sql(name="commits", con=db.conn, if_exists="replace", index=False)

721

In [27]:
# print("url_commits_lab_cnt:", url_commits_cnt)
# print("pulls:", url_pulls_cnt)
# print("issue:", url_issues_cnt)
# print("raws:", url_raws_cnt)
# sum_cnt = url_commits_cnt + url_pulls_cnt + url_issues_cnt + url_raws_cnt
# print(sum_cnt)
# len(df_master)

fixes = pd.read_sql("SELECT * FROM commits", con=db.conn)
print("# vulnerabilities with fix commit:", len(set(fixes['cve_id'])))
print("# fix commit:", len(set(fixes['hash'])))
print(len(fixes))

# vulnerabilities with fix commit: 484
# fix commit: 719
721
